In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.layers import Input, Embedding, LSTM, Dense, Conv2D
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

IMAGE_HEIGHT, IMAGE_WIDTH = 240, 240
TUMOR_IMAGES = 155
df = pd.read_csv("data/survival_data.csv")
df.head()

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7014089426487103501
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 368050176
locality {
  bus_id: 1
}
incarnation: 8878213718973477404
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0"
]


,Brats17ID,Age,Survival
0,Brats17_TCIA_167_1,74.907,153
1,Brats17_TCIA_242_1,66.479,147
2,Brats17_TCIA_319_1,64.860,254
3,Brats17_TCIA_469_1,63.899,519
4,Brats17_TCIA_218_1,57.345,346


In [5]:
X = np.load("tumors_nx.npy")
Y = df['Survival']

### 163 patients with tumor images of sizes 240x240 of which we have 155

In [ ]:
with tf.device('/gpu:0'):
    
    main_input = Input(shape=X.shape[1:], dtype='float32', name='main_input')
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(main_input)
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    cnn_out = Flatten()(x)
    auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(cnn_out)
    
    auxiliary_input = Input(shape=(1,), name='aux_input')
    x = keras.layers.concatenate([cnn_out, auxiliary_input])
    x = Dense(64, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    main_output = Dense(1, activation='sigmoid', name='main_output')(x)
    
    
    model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])
    
    # RMSprop uses:
    # - Momentum taking knowledge from previous steps into account about where
    #   we should be heading (prevents gradient descent to oscillate)
    # - Uses recent gradients to adjust alpha
    #   (when the gradient is very large, alpha is reduced and vice-versa)
    # Later we should test if AdaDelta or Adam are improving our results (quite similar to RMSprop)
    model.compile(optimizer='rmsprop',
              loss={'main_output': 'mean_squared_error', 'aux_output': 'mean_squared_error'},
              loss_weights={'main_output': 1., 'aux_output': 0.2})

    # And trained it via:
    model.fit({'main_input': X, 'aux_input': df['Age']},
              {'main_output': Y, 'aux_output': Y},
              epochs=50, batch_size=32)

Epoch 1/50
163/163 [==============================] - 19s - loss: 359574.2461 - main_output_loss: 299633.3620 - aux_output_loss: 299704.4110    
Epoch 2/50
 64/163 [==========>...................] - ETA: 4s - loss: 316057.8281 - main_output_loss: 263381.5156 - aux_output_loss: 263381.5156

### Start
Implementing a concatenated model as shown in the Docs: https://keras.io/getting-started/functional-api-guide/#multi-input-and-multi-output-models
The model uses a small common CNN for processing one patient (155 slices x 128 width x 128 height) and just inserts the age at a certain point. With this simple model we get really bad results (main_output_loss: 263381.5156 [not getting better]).


### Possible improvements
1. Conv3D (in CNN) which gets a 4D input - therefore add the tumor region as a dimension (transform input image into three images containing each tumor region seperately)
2. One hot encoder (in sklearn or implement manually)
3. Group ages into few classes changing the regression task into a classification task (with softmax).
4. Evaluate if LSTM is useful for this task (before concatenating the two branches or as a concatenator?)
5. Evaluate our optimizer (RMSprp vs Adam vs AdaDelta)
6. I read full batch learning is the best way for training on few training data - check if this is helpful.
7. Use for a patient nx, ny and nz at the same time
8. Data augmentation?